In [1]:
%%capture
!pip install transformers
!pip install wandb

# MTL approach
- Use Soft and Hard Labels
- Hard:
  - Encoding: ~~1_hot~~ / One label 
  - Loss: ~~Cross_entropy~~/ Negative_Log_Like
- Soft
  - Loss: ~~Cross_entropy~~ / KL_div

- Model output layer:
  - ~~1 Layer with 4 outputs (loss with [:2] & [2:])~~
  - 2 seperate headers
  - ~~ 1 header and alternating loss (different loss)~~

- Layers to train
 - ~~ Whole model~~
 - output
 - Specific number of last layers: 1 (pooler + 11th) 

- Regularization:
  - Dropout


**Results**: https://wandb.ai/capture_disagreement/MTL/runs/y6423384?workspace=user-sheuschk

In [ ]:
#TODO: Fit training to 2 loss functions

In [2]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sheuschk (capture_disagreement). Use `wandb login --relogin` to force relogin


True

In [3]:
from drive.MyDrive.cicl_data.helpers import read_data
# from drive.MyDrive.cicl_data.code import CustomLabelDataset

In [4]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
# from datasets import Dataset
import torch.nn.functional as Fun
from torch.utils.data import Dataset, random_split, DataLoader
from torch.optim import AdamW
import torch.nn as nn

from tqdm.notebook import tqdm


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
data_dict = read_data()
df_all = pd.concat([data_dict[k] for k in data_dict.keys()])

### Pretrained model

In [7]:
# Maybe load from wandb in future
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", do_lower_case=True)

In [8]:
from transformers import BertModel
base_model = BertModel.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English")
# class_model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", num_labels=2)

# output of model: https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

Some weights of the model checkpoint at lanwuwei/GigaBERT-v4-Arabic-and-English were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
class MTLModel(nn.Module):
  def __init__(self, base_model):
    super().__init__()
    self.bert = base_model
    self.linear_hl = nn.Linear(768, 2)
    self.linear_sl = nn.Linear(768, 2)
    self.dropout = nn.Dropout(0.1)
    # [8, 240, 768]

  def forward(self, input_ids, attention_mask, token_type_ids):
    """a linear layer on top of the pooled output (https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#bertforsequenceclassification)"""

    x = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    x = self.dropout(x.pooler_output)
    
    x_hl = self.linear_hl(x)
    x_hl = Fun.log_softmax(x_hl, dim=1)

    x_sl = self.linear_sl(x)
    x_sl = Fun.log_softmax(x_sl, dim=1)
    return x_hl, x_sl

In [10]:
model = MTLModel(base_model).to(device)

In [11]:
# Deactivate weights for backprob
for name, param in model.named_parameters():
  if "bert.embeddings" in name:
    param.requires_grad = False
  for i in range(0, 11):  # BERT Model has 11 Bert Layers (only train last one)
    if str(i) in name:
      param.requires_grad = False


## Prepare Data

In [12]:
# from drive.MyDrive.cicl_data.helpers import CustomLabelDataset
# return: input, hard_labels_1hot, soft_labels, hard_labels

In [12]:
class CustomLabelDataset(Dataset):
    def __init__(self, df_all):
        self.text = list(map(self.tokenize_func, df_all["text"]))
        self.soft_labels = df_all["soft_list"] 
        self.hard_labels = df_all["hard_label"]
        self.hard_labels_1h = Fun.one_hot(torch.tensor(df_all['hard_label'].values))

    def __len__(self):
        return len(self.text)
      
    def tokenize_func(self, text):
        return tokenizer(text, padding="max_length", truncation=True, max_length=240)

    def __getitem__(self, idx):
        input = {"attention_mask": torch.tensor(self.text[idx]["attention_mask"]),
                 "token_type_ids": torch.tensor(self.text[idx]["token_type_ids"]),
                 "input_ids": torch.tensor(self.text[idx]["input_ids"])}
        return input, self.hard_labels_1h[idx], torch.tensor(self.soft_labels[idx]), torch.tensor(self.hard_labels[idx])


In [13]:
# Init dataset
dataset = CustomLabelDataset(df_all)
batch_size = 4

train_size = int(len(dataset) * 0.8)
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True)

eval_dataloader = DataLoader(
    eval_dataset,
    batch_size=batch_size)

## Optimization

In [14]:
# Optimizer
num_epochs = 10

num_training_steps = num_epochs * len(train_dataloader)
optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [15]:
# Loss (both expect log probabilities)
loss_hard = nn.NLLLoss()
loss_soft = nn.KLDivLoss(reduction="batchmean")

## Training

In [16]:
run = wandb.init(
    project="MTL",
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "device": device
        },
      save_code = True,
      tags = ["bert_arabic_english", "mtl", "2_heads", "11Bert_and_classification", "NLL_and_KL_loss", "10epochs"],
      )
wandb.watch(model, log_freq=100)

[]

In [17]:
# from drive.MyDrive.cicl_data.helpers import ce_eval_func
def ce_eval_func(model, eval_dataloader, eval_size, epsilon=1e-12, device="cuda"):
  model.eval()
  cross_error = 0

  for i, batch in enumerate(tqdm(eval_dataloader, 0)):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    soft_labels = batch[2].to(device)

    with torch.no_grad():
      _, pred_sl = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    probabilities = torch.softmax(pred_sl, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)
    cross_error += -torch.sum(soft_labels * torch.log(predictions + 1e-9))

  return cross_error / eval_size



In [18]:
# Train
# num_epochs = wandb.config.epochs

for e in range(num_epochs):
  model.train()
  loss_batches = 0
  epoch_loss = 0
  epoch_len = len(train_dataloader)

  for i, batch in enumerate(train_dataloader):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    hard_labels, soft_labels = batch[3].to(device), batch[2].to(device)

    # predict
    optimizer.zero_grad()
    pred_hl, pred_sl = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    # Loss
    loss_hl = loss_hard(pred_hl, hard_labels)
    loss_sl = loss_soft(pred_sl, soft_labels)

    loss = loss_hl + loss_sl  #TODO Weight loss (HL is greater)

    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # Log
    loss_batches += loss.item()
    epoch_loss += loss.item()
    wandb.log({"train/loss": loss.item()})

    log_n_batches = 200
    if i % log_n_batches == 0:
      if i != 0:
        print(f"{e+1}: Last {log_n_batches} batches avg loss: {loss_batches/log_n_batches:>7f}  [{i}/{epoch_len}]")
        wandb.log({"train/loss_over_batches": loss_batches/log_n_batches})
        wandb.log({"train/epochs": e})
      loss_batches = 0
  
  epoch_loss /= i  # Not completely correct (Loss per batch but not every batch has same size)
  print(f"Epoch [{e+1}/{num_epochs}] mean loss: {epoch_loss:>6f}")
  wandb.log({"train/epoch_loss": epoch_loss})

  # Eval error
  ce = ce_eval_func(model, eval_dataloader, eval_size, device=device)
  print(f"Epoch [{e+1}/{num_epochs}] Eval CE  : {ce:>6f}")
  wandb.log({"eval/epoch_ce": ce})



1: Last 200 batches avg loss: 1.023856  [200/2086]
1: Last 200 batches avg loss: 0.922845  [400/2086]
1: Last 200 batches avg loss: 0.948020  [600/2086]
1: Last 200 batches avg loss: 0.978440  [800/2086]
1: Last 200 batches avg loss: 0.921321  [1000/2086]
1: Last 200 batches avg loss: 0.909566  [1200/2086]
1: Last 200 batches avg loss: 0.934567  [1400/2086]
1: Last 200 batches avg loss: 0.944141  [1600/2086]
1: Last 200 batches avg loss: 0.962150  [1800/2086]
1: Last 200 batches avg loss: 0.893795  [2000/2086]
Epoch [1/10] mean loss: 0.942070


  0%|          | 0/522 [00:00<?, ?it/s]

Epoch [1/10] Eval CE  : 0.616592
2: Last 200 batches avg loss: 0.871573  [200/2086]
2: Last 200 batches avg loss: 0.968960  [400/2086]
2: Last 200 batches avg loss: 0.943335  [600/2086]
2: Last 200 batches avg loss: 0.928893  [800/2086]
2: Last 200 batches avg loss: 0.903576  [1000/2086]
2: Last 200 batches avg loss: 0.914128  [1200/2086]
2: Last 200 batches avg loss: 0.823448  [1400/2086]
2: Last 200 batches avg loss: 0.889901  [1600/2086]
2: Last 200 batches avg loss: 0.836441  [1800/2086]
2: Last 200 batches avg loss: 0.866682  [2000/2086]
Epoch [2/10] mean loss: 0.894027


  0%|          | 0/522 [00:00<?, ?it/s]

Epoch [2/10] Eval CE  : 0.577825
3: Last 200 batches avg loss: 0.925844  [200/2086]
3: Last 200 batches avg loss: 0.877755  [400/2086]
3: Last 200 batches avg loss: 0.905820  [600/2086]
3: Last 200 batches avg loss: 0.842366  [800/2086]
3: Last 200 batches avg loss: 0.819292  [1000/2086]
3: Last 200 batches avg loss: 0.896592  [1200/2086]
3: Last 200 batches avg loss: 0.869993  [1400/2086]
3: Last 200 batches avg loss: 0.951405  [1600/2086]
3: Last 200 batches avg loss: 0.866619  [1800/2086]
3: Last 200 batches avg loss: 0.792716  [2000/2086]
Epoch [3/10] mean loss: 0.875172


...

  0%|          | 0/522 [00:00<?, ?it/s]

Epoch [6/10] Eval CE  : 0.527225
7: Last 200 batches avg loss: 0.771361  [200/2086]
7: Last 200 batches avg loss: 0.812639  [400/2086]
7: Last 200 batches avg loss: 0.766547  [600/2086]
7: Last 200 batches avg loss: 0.773877  [800/2086]
7: Last 200 batches avg loss: 0.811406  [1000/2086]
7: Last 200 batches avg loss: 0.714164  [1200/2086]
7: Last 200 batches avg loss: 0.775826  [1400/2086]
7: Last 200 batches avg loss: 0.740724  [1600/2086]
7: Last 200 batches avg loss: 0.783912  [1800/2086]
7: Last 200 batches avg loss: 0.796016  [2000/2086]
Epoch [7/10] mean loss: 0.774559


  0%|          | 0/522 [00:00<?, ?it/s]

Epoch [7/10] Eval CE  : 0.542731


KeyboardInterrupt: ignored

Model dependent improvements:


## Evaluation

In [19]:
# initialize Evaluation dataset
data_dict_dev = read_data("dev")
df_dev = pd.concat([data_dict_dev[k] for k in data_dict_dev.keys()])

dev_dataset = CustomLabelDataset(df_dev)
dev_batch_size = 4
dev_size = len(dev_dataset)

dev_dataloader = DataLoader(
    dev_dataset,
    batch_size=dev_batch_size)

In [20]:
# Cross Entropy Error
cross_error = ce_eval_func(model, dev_dataloader, dev_size, device=device)
print(f"CE error: {cross_error}")
wandb.log({"dev/ce": cross_error})

  0%|          | 0/557 [00:00<?, ?it/s]

CE error: 0.5314700603485107


In [21]:
# from drive.MyDrive.cicl_data.helpers import f1_eval_func
from sklearn.metrics import f1_score

def f1_eval_func(model, eval_dataloader, eval_size, device="cuda"):
  model.eval()
  f1_error = 0

  for i, batch in enumerate(tqdm(eval_dataloader, 0)):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    hard_labels = batch[3].to(device)

    with torch.no_grad():
      _, pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    f1_error += f1_score(hard_labels.cpu(), pred.argmax(1).cpu(), average='micro')

  return f1_error / eval_size

In [22]:
# F1 micro Error
f1 = f1_eval_func(model, dev_dataloader, dev_size, device)
print(f"F1 error: {f1}")
wandb.log({"dev/f1": f1})

  0%|          | 0/557 [00:00<?, ?it/s]

F1 error: 0.18820224719101122


In [23]:
model.eval()
cross_error = 0
epsilon = 1e-12
for i, batch in enumerate(eval_dataloader):
  input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
  attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
  token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
  soft_labels = batch[2].to(device)

  with torch.no_grad():
    _, pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
  probabilities = torch.softmax(pred, axis=-1)
  predictions = torch.clip(probabilities, epsilon, 1. - epsilon)
  cross_error += -torch.sum(soft_labels * torch.log(predictions + 1e-9))
  break



In [24]:
print(predictions)
print(soft_labels)

tensor([[0.7617, 0.2383],
        [0.6778, 0.3222],
        [0.7249, 0.2751],
        [0.8924, 0.1076]], device='cuda:0')
tensor([[0.4000, 0.6000],
        [0.3300, 0.6700],
        [0.0000, 1.0000],
        [0.0000, 1.0000]], device='cuda:0')


### Finish

In [ ]:
# Save parameters


In [25]:
torch.save(model.parameters, 'model.pt')
artifact = wandb.Artifact(name='model_param', type='model')
artifact.add_file(local_path="model.pt")
run.log_artifact(artifact);

In [26]:
wandb.finish()

dev/ce,▁
dev/f1,▁
eval/epoch_ce,█▅▇▁▁▂▃
train/epoch_loss,█▆▅▃▃▁▁
train/epochs,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇██████
train/loss,▄▂▇▂▃▁▂▃▆▃▂▂▅█▅▆▁▄▃▁▆▂▂▄▂▁▁▅▂▁▃▁▄▅▃▄▂▂▃▃
train/loss_over_batches,█▆▇▅▆▇▅▆▅▆▅▄▅▄▃▅▄▄▃▃▄▄▄▄▃▄▃▃▃▂▂▂▂▃▂▃▂▁▂▃
dev/ce,0.53147
dev/f1,0.1882
eval/epoch_ce,0.54273
train/epoch_loss,0.77456


## TSV files

In [ ]:
import os
import csv

In [ ]:
filepaths = ["/content/ArMIS_results.tsv", "/content/ConvAbuse_results.tsv", "/content/HS-Brexit_results.tsv", "/content/MD-Agreement_results.tsv"]
epsilon = 1e-12

for fp in filepaths:
  if os.path.exists(fp):
    os.remove(fp)

for key in data_dict_dev.keys():
  tsv_dataset = CustomLabelDataset(data_dict_dev[key])
  tsv_dataloader = DataLoader(tsv_dataset, shuffle=False, batch_size=1)
  filepath_write = f"/content/{key}_results.tsv"

  with open(filepath_write, 'w', newline='') as tsvfile:
      writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
      for i, batch in enumerate(tqdm(tsv_dataloader, 0)):
        input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
        attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
        token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)

        with torch.no_grad():
          pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = pred.logits
        prediction = torch.argmax(logits, dim=-1)
        probability = torch.softmax(logits, axis=-1)
        probability = torch.clip(probability, epsilon, 1. - epsilon) # Really necessary?
        writer.writerow([prediction[0].item(), probability[0][0].item(), probability[0][1].item()])


In [ ]:
from zipfile import ZipFile

filepath = "res.zip" 

if os.path.exists(filepath):
    os.remove(filepath)

#loop over filepath names throws an string index out of range for whatever reason(also can't use content here, not sure why)
with ZipFile(filepath, 'w') as zipObj:
  zipObj.write("MD-Agreement_results.tsv")
  zipObj.write("ArMIS_results.tsv")
  zipObj.write("HS-Brexit_results.tsv")
  zipObj.write("ConvAbuse_results.tsv")

In [ ]:
from google.colab import files
files.download("res.zip")